In [492]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [493]:
# Load the data into a Pandas DataFrame and make the index the "coin_id" column.

market_data_df = pd.read_csv("Resources/crypto_market_data.csv", index_col="coin_id")
# Display sample data
market_data_df.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [494]:
# Generate summary statistics
market_data_df.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


### Prepare the Data

In [495]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
         
scaled_data=StandardScaler().fit_transform(market_data_df)
        

In [496]:
# Create a DataFrame with the scaled data

type(scaled_data)


# Copy the crypto names from the original data # Set the coinid column as index

scaled_data_df = pd.DataFrame(scaled_data, index=market_data_df.index, columns=market_data_df.columns)

# Display sample data
scaled_data_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


### Find the Best Value for k Using the Original Scaled DataFrame.

In [497]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11


k_list= list(range(1, 11))

# Create an empty list to store the inertia values, 
   
inertia_list=[]

# Create a for loop to compute the inertia with each possible value of k 
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using the scaled DataFrame
# 3. Append the model.inertia_ to the inertia list

 
for k in k_list:
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=1) ## gives same initial groups everytime... want my answer to be same as Instructor
    kmeans.fit(scaled_data_df)
    inertia_list.append(kmeans.inertia_)



# Create a dictionary with the data to plot the Elbow curve  first look at the list inertia_list then Xpert learn 


elbow_data = {
    "k": k_list,
    "inertia": inertia_list
}
# Create a DataFrame with the data to plot the Elbow curve
elbow_data_df = pd.DataFrame(elbow_data)

# Display the DataFrame
elbow_data_df

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,165.136752
3,4,79.022435
4,5,66.413051
5,6,61.634088
6,7,46.074467
7,8,43.392694
8,9,32.913223
9,10,30.469111


In [498]:
#Plot a line chart with all the inertia values computed with 

elbow_data_curve=elbow_data_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
elbow_data_curve

:Curve   [k]   (inertia)

#### Answer the following question: 
**Question:** What is the best value for `k`?

**Answer:**

The best value for K is 4. 

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data.

In [499]:
# Initialize the K-Means model using the best value for k

k1_model= KMeans(n_clusters=4, n_init='auto',random_state=1)

In [500]:
# Fit the K-Means model using the scaled data

k1_model.fit(scaled_data_df)

KMeans(n_clusters=4, random_state=1)

In [501]:
# Predict the clusters to group the cryptocurrencies using the scaled data

clusters = k1_model.predict(scaled_data_df)
# View the resulting array of cluster values.
clusters

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [502]:
# Create a copy of the DataFrame
predicited_scaled_data =scaled_data_df.copy()

In [503]:
# Add a new column to the DataFrame with the predicted clusters

predicited_scaled_data["Predicted Clusters"] = clusters

# Display sample data
predicited_scaled_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Predicted Clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [504]:
# Create a scatter plot using Pandas plot by setting 

scatter_pca_predictions= predicited_scaled_data.hvplot.scatter(
    title= "Price Change 24h vs 7d",
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="Predicted Clusters",
    colormap="rainbow"
  
)
scatter_pca_predictions

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,Predicted Clusters)

### Optimize Clusters with Principal Component Analysis.

In [505]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [506]:
# Use the PCA model with `fit_transform` on the ORIGINAL scaled DataFrame to reduce to three principal components.

pca_df = pca.fit_transform(scaled_data_df)

# View the first five rows of the DataFrame. 

pca_df[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [507]:
# Retrieve the explained variance to determine how much information  can be attributed to each principal component.

pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:**  
pca.explained_variance_ratio_.sum() so answer is 0.8950316570309842

To answer this question we need to find the sum using the code pca.explained_variance_ratio_.sum()
see below. The total explained variance is 89.50% (round up two decimals).

In [508]:
#THIS IS HOW MUCH OF THE ORIGINAL DATA THAT WAS RETAINED SO LOST ABOUT 9% of the DATA -PCA, PCL , LDA ARE  FEATURE REDUCTION TECHNIQUES INTERVIEW QUESTION
pca.explained_variance_ratio_.sum()

0.8950316570309842

In [509]:
# Create a new DataFrame with the PCA data.

pca_data= pd.DataFrame(
    pca_df,
    columns =["PC1", "PC2", "PC3"])

# Copy the crypto names from the ORIGINAL DATA
pca_data["coin_id"]=scaled_data_df.index

# Set the coinid column as index
pca_data=pca_data.set_index("coin_id")

# Display sample data
pca_data.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


### Find the Best Value for k Using the PCA Data

In [510]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k2_list= list(range(1,11))

# Create an empty list to store the inertia values
inertia2_list= []
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using PCA DataFrame.
# 3. Append the model.inertia_ to the inertia list

for k2 in k2_list:
    kmeans = KMeans(n_clusters=k2, n_init='auto', random_state=1)
    kmeans.fit(pca_data)
    inertia2_list.append(kmeans.inertia_)


# Create a dictionary with the data to plot the Elbow curve
elbow_data2= {
    "k2": k2_list,
    "inertia 2": inertia2_list
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_data2_df = pd.DataFrame(elbow_data2)

# Display the DataFrame
elbow_data2_df

,k2,inertia 2
0,1,256.874086
1,2,182.339530
2,3,135.442408
3,4,49.665497
4,5,38.672582
5,6,34.507577
6,7,23.728365
7,8,18.574523
8,9,15.572672
9,10,11.407667


In [511]:
# Plot a line chart with all the inertia values computed with 
import hvplot.pandas
elbow_data2 = elbow_data2_df.hvplot.line(
    x="k2", 
    y="inertia 2", 
    title="Elbow Curve 2", 
    xticks=k2
)
elbow_data2
# the different values of k to visually identify the optimal value for k.


:Curve   [k2]   (inertia 2)

#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer: 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer: no

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [512]:
# Initialize the K-Means model using the best value for k

k2_model= KMeans(n_clusters=4, n_init='auto',random_state=1)


In [513]:
# Fit the K-Means model using the PCA data

k2_model.fit(pca_data)

KMeans(n_clusters=4, random_state=1)

In [514]:
# Predict the clusters to group the cryptocurrencies using the PCA data

clusters2 = k2_model.predict(pca_data)
clusters2

# Print the resulting array of cluster values.


array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [515]:
# Create a copy of the DataFrame with the PCA data

predicited_pca_data =pca_data.copy()

# Add a new column to the DataFrame with the predicted clusters

predicited_pca_data["Predicted Clusters"] = clusters2

# Display sample data
predicited_pca_data.head()

,PC1,PC2,PC3,Predicted Clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [516]:
# Create a scatter plot using hvPlot by setting `x="PCA1"` and `y="PCA2"`. 

scatter_pca_predictions= predicited_pca_data.hvplot.scatter(
    title= "Price Change 24h vs 7d",
    x="PC1",
    y="PC2",
    c="Predicted Clusters",
    colormap="rainbow"
  
)
scatter_pca_predictions

:Scatter   [PC1]   (PC2,Predicted Clusters)

### Determine the Weights of Each Feature on each Principal Component

In [517]:
# Use the columns from the original scaled DataFrame as the index.
scaled_data_df.columns

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')

In [518]:
# Assuming pca is your PCA object and scaled_data_df is your DataFrame after scaling and PCA
# Get the weights of each feature on each principal component
weights = pca.components_

# Create a DataFrame to show the weights of each feature for each principal component
weights_df = pd.DataFrame(weights, columns=scaled_data_df.columns)

# Display the DataFrame
print(weights_df)

   price_change_percentage_24h  price_change_percentage_7d  \
0                    -0.416728                   -0.102432   
1                     0.358162                    0.227432   
2                    -0.218795                    0.787670   

   price_change_percentage_14d  price_change_percentage_30d  \
0                    -0.009162                     0.191523   
1                     0.540415                     0.562182   
2                     0.349534                    -0.181744   

   price_change_percentage_60d  price_change_percentage_200d  \
0                     0.320365                      0.594468   
1                     0.434040                      0.027735   
2                    -0.361377                      0.039733   

   price_change_percentage_1y  
0                    0.568379  
1                   -0.150789  
2                    0.210541  


#### Answer the following question: 

* **Question:** Which features have the strongest positive or negative influence on each component? 
 
* **Answer:** 
We can analyze which features have the strongest positive or negative influence on each component:

Principal Component 0:

Strongest positive influences: price_change_percentage_200d, price_change_percentage_1y
Strongest negative influence: price_change_percentage_24h
Principal Component 1:

Strongest positive influences: price_change_percentage_7d, price_change_percentage_14d, price_change_percentage_30d, price_change_percentage_60d, price_change_percentage_200d
Strongest negative influence: price_change_percentage_1y
Principal Component 2:

Strongest positive influences: price_change_percentage_7d, price_change_percentage_14d, price_change_percentage_1y
Strongest negative influence: price_change_percentage_24h, price_change_percentage_30d, price_change_percentage_60d
    